## קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### העלאת משתנים להרצת הקוד

In [5]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

sys.path.append(future_folder)

## פונקציות

In [3]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [4]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [5]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [6]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [7]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


## חיבור טבלאות אוכלוסייה

In [9]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [10]:
year=['2025','2030','2035','2040','2045','2050']
add_year=['2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045',
 '2045_2050']

In [11]:
v_date='230720'

In [12]:
for y,y1 in zip(year,add_year):  

    folder_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\jew_pop'

    new_column_names = {'aprt_{}'.format(y): 'aprt', 'pop_{}'.format(y): 'pop'}

    file_name='{}_forecast_pop_jewish_{}_jtmt'.format(v_date,y1)
    
    col_for_jew_df=['Taz_num',
                     'pop_0',
                     'pop_10',
                     'pop_15',
                     'pop_20',
                     'pop_25',
                     'pop_30',
                     'pop_35',
                     'pop_40',
                     'pop_45',
                     'pop_5',
                     'pop_50',
                     'pop_55',
                     'pop_60',
                     'pop_65',
                     'pop_70',
                     'pop_75up',
                     'pop_{}'.format(y),
                     'aprt_{}'.format(y)]

    col=['Taz_num', 'aprt','pop_0', 'pop_10', 'pop_15', 'pop_20', 'pop_25', 'pop_30', 'pop_35', 'pop_40', 'pop_45', 'pop_5', 'pop_50', 'pop_55', 'pop_60', 'pop_65', 'pop_70', 'pop_75up', 'pop']
    forecast_pop_jewish=up_load_df(folder_path,file_name)[col_for_jew_df].rename(columns=new_column_names)[col]

    folder_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\arab_pop'
    file_name='taz_Arab_Palestinian_{}'.format(y)

    new_column_names = {'hh_{}'.format(y): 'aprt', 'pop_{}'.format(y): 'pop'}
    forecast_pop_arab=up_load_df(folder_path,file_name).rename(columns=new_column_names)
    
    folder_path_save=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\join_pop'


    pd.concat([forecast_pop_arab,forecast_pop_jewish],axis=0).to_excel('{}\{}_pop_{}_jtmt.xlsx'.format(folder_path_save,file_date,y),index=False)